<font size='5' face='Courier New'><h1 align="center"><i>The Primal & Dual Linear Programming Problems: Canonical Form</i></h1></font> 

<font face='Times New Roman' size='6'><h3 align="center"><u>*James&nbsp;D.&nbsp;Gaboardi*</u></h3></font>

------
<font face='Times New Roman' size='5'><h3 align="center">*Florida State University* &nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp; *Department of Geography*</h3></font>

------

------

<p><font size='4' face='Times New Roman'>Adapted from:</font></p>
<p><font size='4' face='Times New Roman'><b>Daskin, M. S.</b> 1995. <i>Network and Discrete Location: Models, Algorithms, and Applications</i>. Hoboken, NJ, USA: John Wiley & Sons, Inc.</font></p>

-----------

<font size='7' face='Times New Roman'><b>0. <u>Imports and Data Creation</u></b></font>

In [1]:
# Imports
import numpy as np
import gurobipy as gbp
import datetime as dt

#  Constants
Aij = np.random.randint(5, 50, 400)
Aij = Aij.reshape(20,20)
AijSum = np.sum(Aij)
Cj = np.random.randint(10, 20, 20)
CjSum = np.sum(Cj)
Bi = np.random.randint(10, 20, 20)
BiSum = np.sum(Bi)

# Matrix Shape
rows = range(len(Aij))
cols = range(len(Aij[0]))


<font size='7' face='Times New Roman'><b>1. <u>Primal</u></b></font>

In [2]:
def GbpPrimCan():    
    # Instantiate Model
    mPrimal_Canonical_GUROBI = gbp.Model(' -- Canonical Primal Linear Programming Problem -- ')

    # Set Focus to Optimality
    gbp.setParam('MIPFocus', 2)

    # Decision Variables
    desc_var = []
    for dest in cols:
        desc_var.append([])
        desc_var[dest].append(mPrimal_Canonical_GUROBI.addVar(vtype=gbp.GRB.CONTINUOUS, 
                                        name='y'+str(dest+1)))
    # Update Model
    mPrimal_Canonical_GUROBI.update()

    #Objective Function
    mPrimal_Canonical_GUROBI.setObjective(gbp.quicksum(Cj[dest]*desc_var[dest][0] 
                            for dest in cols), 
                            gbp.GRB.MINIMIZE)
    # Constraints
    for orig in rows:
        mPrimal_Canonical_GUROBI.addConstr(gbp.quicksum(Aij[orig][dest]*desc_var[dest][0] 
                            for dest in cols) - Bi[orig] >= 0)
    # Optimize
    try:
        mPrimal_Canonical_GUROBI.optimize()
    except Exception as e:
        print '   ################################################################'
        print ' < ISSUE : ', e, ' >'
        print '   ################################################################'
    # Write LP file
    mPrimal_Canonical_GUROBI.write('LP.lp')
    print '\n*************************************************************************'
    print '    |   Decision Variables'
    for v in mPrimal_Canonical_GUROBI.getVars():
        print '    |  ', v.VarName, '=', v.x
    print '*************************************************************************'
    val = mPrimal_Canonical_GUROBI.objVal
    print '    |   Objective Value ------------------ ', val
    print '    |   Aij Sum -------------------------- ', AijSum
    print '    |   Cj Sum --------------------------- ', CjSum
    print '    |   Bi Sum --------------------------- ', BiSum
    print '    |   Matrix Dimensions ---------------- ', Aij.shape
    print '    |   Date/Time ------------------------ ', dt.datetime.now()
    print '*************************************************************************'
    print '-- Gurobi Canonical Primal Linear Programming Problem --'
    
    
try:
    GbpPrimCan()
    print '\nJames Gaboardi, 2015'
except Exception as e:
    print '   ################################################################'
    print ' < ISSUE : ', e, ' >'
    print '   ################################################################'

Changed value of parameter MIPFocus to 2
   Prev: 0   Min: 0   Max: 3   Default: 0
Optimize a model with 20 rows, 20 columns and 400 nonzeros
Coefficient statistics:
  Matrix range    [5e+00, 5e+01]
  Objective range [1e+01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 20 rows, 20 columns, 400 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.687500e+00   0.000000e+00      0s
      11    7.4815903e+00   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective  7.481590294e+00

*************************************************************************
    |   Decision Variables
    |   y1 = 0.0318421970971
    |   y2 = 0.0
    |   y3 = 0.0687163240813
    |   y4 = 0.0173262243046
    |   y5 = 0.0
    |   y6 = 0.0
    |   y7 = 0.00578178597961
    |   y8 = 0.0
    |   y9 = 0.0270323672164
    |   y10 = 0.0
    |   y11 = 0.0
    |   y12 = 

<font size='7' face='Times New Roman'><b>2. <u>Dual</u></b></font>

In [11]:
def GbpDualCan():    
    # Instantiate Model
    mDual_Canonical_GUROBI = gbp.Model(' -- Canonical Dual Linear Programming Problem -- ')

    # Set Focus to Optimality
    gbp.setParam('MIPFocus', 2)

    # Decision Variables
    desc_var = []
    for dest in cols:
        desc_var.append([])
        desc_var[dest].append(mDual_Canonical_GUROBI.addVar(vtype=gbp.GRB.CONTINUOUS, 
                                        name='u'+str(dest+1)))
    # Update Model
    mDual_Canonical_GUROBI.update()

    #Objective Function
    mDual_Canonical_GUROBI.setObjective(gbp.quicksum(Bi[orig]*desc_var[orig][0] 
                            for orig in rows), 
                            gbp.GRB.MAXIMIZE)
    # Constraints
    for dest in cols:
        mDual_Canonical_GUROBI.addConstr(gbp.quicksum(Aij[orig][dest]*desc_var[dest][0] 
                            for orig in rows) - Cj[dest] <= 0)
    # Optimize
    mDual_Canonical_GUROBI.optimize()
    # Write LP file
    mDual_Canonical_GUROBI.write('LP.lp')
    print '\n*************************************************************************'
    print '    |   Decision Variables'
    for v in mDual_Canonical_GUROBI.getVars():
        print '    |  ', v.VarName, '=', v.x
    print '*************************************************************************'
    val = mDual_Canonical_GUROBI.objVal
    print '    |   Objective Value ------------------ ', val
    print '    |   Aij Sum -------------------------- ', AijSum
    print '    |   Cj Sum --------------------------- ', CjSum
    print '    |   Bi Sum --------------------------- ', BiSum
    print '    |   Matrix Dimensions ---------------- ', Aij.shape
    print '    |   Date/Time ------------------------ ', dt.datetime.now()
    print '*************************************************************************'
    print '-- Gurobi Canonical Dual Linear Programming Problem --'
    print '\nJames Gaboardi, 2015'
    
try:

Parameter MIPFocus unchanged
   Value: 2   Min: 0   Max: 3   Default: 0
Optimize a model with 500 rows, 500 columns and 500 nonzeros
Coefficient statistics:
  Matrix range    [1e+04, 1e+04]
  Objective range [1e+01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 2e+01]
Presolve removed 500 rows and 500 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.7435250e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  7.743525048e+00

*************************************************************************
    |   Decision Variables
    |   u1 = 0.00119886108197
    |   u2 = 0.00135389244077
    |   u3 = 0.00124033270101
    |   u4 = 0.00128797636185
    |   u5 = 0.00128783000644
    |   u6 = 0.000896793961587
    |   u7 = 0.00128612498109
    |   u8 = 0.00140813755281
    |   u9 = 0.00105310666466
    |   u10 = 0.0007966396292
  